In [12]:
# Step 1: Install packages

# %pip install pandas scikit-learn
# %pip install transformers torch
# %pip install --upgrade jupyter ipywidgets
# %pip install nltk
# %pip install matplotlib


In [13]:
import nltk

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nikolay_tishchenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nikolay_tishchenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nikolay_tishchenko/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
# Step 2: Import Libraries
import os
import platform

import pandas as pd
import numpy as np

import pyodbc
import matplotlib.pyplot as plt

import torch
from transformers import pipeline, BertTokenizer, BertModel, AutoTokenizer


from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import re

from collections import Counter
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
import textwrap
from nltk.corpus import wordnet

# Check if 'wordnet' and 'averaged_perceptron_tagger' are already downloaded, and download them if necessary
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)


True

### Identify action verbs and related nouns to build 'root causes'

In [15]:
# Import NLTK and download necessary resources
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

In [16]:
# Download necessary NLTK resources
nltk_data_path = '/Users/nikolay_tishchenko/nltk_data'
nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('wordnet', download_dir=nltk_data_path)
nltk.download('averaged_perceptron_tagger', download_dir=nltk_data_path)

# Verify the resources have been downloaded
print(nltk.data.path)   # Print the NLTK data path


['/Users/nikolay_tishchenko/nltk_data', '/Users/nikolay_tishchenko/bitbucket_projects/coolsys/.venv/nltk_data', '/Users/nikolay_tishchenko/bitbucket_projects/coolsys/.venv/share/nltk_data', '/Users/nikolay_tishchenko/bitbucket_projects/coolsys/.venv/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nikolay_tishchenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nikolay_tishchenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nikolay_tishchenko/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
import nltk

# Define the custom path for nltk_data
nltk_data_path = '/Users/nikolay_tishchenko/nltk_data'

# Add the custom path to nltk.data.path
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)

# Verify the custom path is added
print(nltk.data.path)

['/Users/nikolay_tishchenko/nltk_data', '/Users/nikolay_tishchenko/bitbucket_projects/coolsys/.venv/nltk_data', '/Users/nikolay_tishchenko/bitbucket_projects/coolsys/.venv/share/nltk_data', '/Users/nikolay_tishchenko/bitbucket_projects/coolsys/.venv/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [18]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

# Define the text to analyze
text = """
performed pm per scope hvac tasks replaced air filters visually inspected for refrigerant leaks lubricated all bearings as needed shut off water supply  
fall visit as needed  inspected belts and adjusted as needed cleaned condensate lines checked contactors visually inspected evaporator and condenser coils 
inspected and tested heating circuit for operation inspected and tested each condensate pump if applicable refrigerations tasks inspected all hinges and 
gaskets checked electrical contacts  controls and components inspected relays and contactors visually inspected for leaks cleaned all pre filter media and 
replaced if applicable brushed coil fins as needed inspected blower wheels and fans lubricated motors and bearings as needed checked controls calibration and 
operation walk in tasks inspecfted all hinges and gaskets checked electrical contacts controls and components inspected relays and contactors visually inspected 
for leaks cleane dall pre filter media and replaced if applicable brushed coil fins as needed inspected blower wheels and fans lubricated motors and bearings 
inspected sight glass checked control calibration and operation ice machine tasks checked ice production thickness and sensing probes inspected door and gasket 
visually inspected bin and bin tstat inspected water pumps and distribution tubes visually inspected water filters inspected for any leaks cleaned condenser coils 
inspected all electrical components inspected fan motor  blades   bearings   lubricated as needed descaled ice machine exhaust fan tasks inspected roof fan belts and 
sheaves adjusted as needed replaced belts as needed  20x25x2 pleated filter high efficiency                                                                                                                                                       2023 Today  I arrived on site at unit IM01A and found that it was not operating  After troubleshooting  I discovered the following issues     Unit was off with a low water alarm    Water fill valve needs to be replaced   To resolve these issues  the following work was performed     Replaced the water fill valve    Replaced the water regulator    Checked over all operations and monitored the system   I am happy to report that the unit is now working normally  Upon departure  the unit was operating as expected  Solenoid  Water Feed 115V
"""

# Tokenize the text
tokens = word_tokenize(text)

# Perform POS tagging
tagged_tokens = pos_tag(tokens)

# Extract verbs from the POS tagged tokens
action_verbs = {word for word, pos in tagged_tokens if pos.startswith('VB')}

# Verify that all values in action_verbs are actually verbs using nltk
def is_verb(word):
    synsets = wordnet.synsets(word)
    if not synsets:
        return False
    return any(ss.pos() == 'v' for ss in synsets)

# Filter out non-verbs
verified_action_verbs = [verb for verb in action_verbs if is_verb(verb)]

# Print the identified action verbs
print("Identified action verbs:")
print(list(action_verbs))

# Print the verified action verbs
print("Verified action verbs:")
print(verified_action_verbs)

Identified action verbs:
['brushed', 'arrived', 'operating', 'Checked', 'needed', 'discovered', 'Feed', 'Replaced', 'working', 'exhaust', 'was', 'be', 'checked', 'am', 'performed', 'cleane', 'inspecfted', 'monitored', 'inspected', 'walk', 'is', 'found', 'tested', 'sensing', 'report', 'replaced', 'resolve', 'troubleshooting', 'adjusted', 'lubricated', 'expected', 'cleaned']
Verified action verbs:
['brushed', 'arrived', 'operating', 'Checked', 'needed', 'discovered', 'Feed', 'Replaced', 'working', 'exhaust', 'was', 'be', 'checked', 'am', 'performed', 'monitored', 'inspected', 'walk', 'is', 'found', 'tested', 'sensing', 'report', 'replaced', 'resolve', 'troubleshooting', 'adjusted', 'lubricated', 'expected', 'cleaned']


In [19]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Define the text to analyze
text = """
performed pm per scope hvac tasks replaced air filters visually inspected for refrigerant leaks lubricated all bearings as needed shut off water supply  
fall visit as needed  inspected belts and adjusted as needed cleaned condensate lines checked contactors visually inspected evaporator and condenser coils 
inspected and tested heating circuit for operation inspected and tested each condensate pump if applicable refrigerations tasks inspected all hinges and 
gaskets checked electrical contacts  controls and components inspected relays and contactors visually inspected for leaks cleaned all pre filter media and 
replaced if applicable brushed coil fins as needed inspected blower wheels and fans lubricated motors and bearings as needed checked controls calibration and 
operation walk in tasks inspecfted all hinges and gaskets checked electrical contacts controls and components inspected relays and contactors visually inspected 
for leaks cleane dall pre filter media and replaced if applicable brushed coil fins as needed inspected blower wheels and fans lubricated motors and bearings 
inspected sight glass checked control calibration and operation ice machine tasks checked ice production thickness and sensing probes inspected door and gasket 
visually inspected bin and bin tstat inspected water pumps and distribution tubes visually inspected water filters inspected for any leaks cleaned condenser coils 
inspected all electrical components inspected fan motor  blades   bearings   lubricated as needed descaled ice machine exhaust fan tasks inspected roof fan belts and 
sheaves adjusted as needed replaced belts as needed  20x25x2 pleated filter high efficiency                                                                                                                                                       2023 Today  I arrived on site at unit IM01A and found that it was not operating  After troubleshooting  I discovered the following issues     Unit was off with a low water alarm    Water fill valve needs to be replaced   To resolve these issues  the following work was performed     Replaced the water fill valve    Replaced the water regulator    Checked over all operations and monitored the system   I am happy to report that the unit is now working normally  Upon departure  the unit was operating as expected  Solenoid  Water Feed 115V
"""

# Tokenize the text
tokens = word_tokenize(text)

# Perform POS tagging
tagged_tokens = pos_tag(tokens)

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to convert POS tag to WordNet POS tag
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# Extract verbs from the POS tagged tokens and lemmatize them
action_verbs = set()
for word, pos in tagged_tokens:
    wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
    lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
    if wordnet_pos == wordnet.VERB:
        action_verbs.add(lemma)

# Verify that all values in action_verbs are actually verbs using nltk
def is_verb(word):
    synsets = wordnet.synsets(word, pos=wordnet.VERB)
    return bool(synsets)

# Filter out non-verbs
verified_action_verbs = [verb for verb in action_verbs if is_verb(verb)]

# Print the identified action verbs
print("Identified action verbs:")
print(list(action_verbs))

# Print the verified action verbs
print("Verified action verbs:")
print(verified_action_verbs)

Identified action verbs:
['arrive', 'discover', 'troubleshoot', 'sense', 'adjust', 'operate', 'Checked', 'Feed', 'work', 'Replaced', 'lubricate', 'exhaust', 'be', 'need', 'cleane', 'inspecfted', 'monitor', 'perform', 'walk', 'check', 'brush', 'replace', 'clean', 'report', 'resolve', 'test', 'expect', 'inspect', 'find']
Verified action verbs:
['arrive', 'discover', 'troubleshoot', 'sense', 'adjust', 'operate', 'Checked', 'Feed', 'work', 'Replaced', 'lubricate', 'exhaust', 'be', 'need', 'monitor', 'perform', 'walk', 'check', 'brush', 'replace', 'clean', 'report', 'resolve', 'test', 'expect', 'inspect', 'find']


In [23]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Define the text to analyze
text = """
performed pm per scope hvac tasks replaced air filters visually inspected for refrigerant leaks lubricated all bearings as needed shut off water supply  
fall visit as needed  inspected belts and adjusted as needed cleaned condensate lines checked contactors visually inspected evaporator and condenser coils 
inspected and tested heating circuit for operation inspected and tested each condensate pump if applicable refrigerations tasks inspected all hinges and 
gaskets checked electrical contacts  controls and components inspected relays and contactors visually inspected for leaks cleaned all pre filter media and 
replaced if applicable brushed coil fins as needed inspected blower wheels and fans lubricated motors and bearings as needed checked controls calibration and 
operation walk in tasks inspecfted all hinges and gaskets checked electrical contacts controls and components inspected relays and contactors visually inspected 
for leaks cleane dall pre filter media and replaced if applicable brushed coil fins as needed inspected blower wheels and fans lubricated motors and bearings 
inspected sight glass checked control calibration and operation ice machine tasks checked ice production thickness and sensing probes inspected door and gasket 
visually inspected bin and bin tstat inspected water pumps and distribution tubes visually inspected water filters inspected for any leaks cleaned condenser coils 
inspected all electrical components inspected fan motor  blades   bearings   lubricated as needed descaled ice machine exhaust fan tasks inspected roof fan belts and 
sheaves adjusted as needed replaced belts as needed  20x25x2 pleated filter high efficiency                                                                                                                                                       2023 Today  I arrived on site at unit IM01A and found that it was not operating  After troubleshooting  I discovered the following issues     Unit was off with a low water alarm    Water fill valve needs to be replaced   To resolve these issues  the following work was performed     Replaced the water fill valve    Replaced the water regulator    Checked over all operations and monitored the system   I am happy to report that the unit is now working normally  Upon departure  the unit was operating as expected  Solenoid  Water Feed 115V
"""

# Tokenize the text
tokens = word_tokenize(text)

# Perform POS tagging
tagged_tokens = pos_tag(tokens)

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to convert POS tag to WordNet POS tag
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# Extract verbs and their corresponding nouns from the POS tagged tokens
action_verbs = set()
verb_noun_pairs = []
for i, (word, pos) in enumerate(tagged_tokens):
    wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
    lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
    if wordnet_pos == wordnet.VERB:
        action_verbs.add(lemma)
        # Look for the corresponding noun before the verb
        if i > 0 and tagged_tokens[i-1][1].startswith('NN'):
            verb_noun_pairs.append((tagged_tokens[i-1][0], lemma))

# Verify that all values in action_verbs are actually verbs using nltk
def is_verb(word):
    synsets = wordnet.synsets(word, pos=wordnet.VERB)
    return bool(synsets)

# Filter out non-verbs
verified_action_verbs = [verb for verb in action_verbs if is_verb(verb)]

# Combine verified action verbs with corresponding nouns
verified_verb_noun_pairs = [(verb, noun) for verb,noun in verb_noun_pairs if verb in verified_action_verbs]

# Print the identified action verbs
print("Identified action verbs:")
print(list(action_verbs))

# Print the verified action verbs
print("Verified action verbs:")
print(verified_action_verbs)

# Print the verified noun-verb pairs
print("Verified noun-verb pairs:")
print(verified_noun_verb_pairs)

Identified action verbs:
['arrive', 'discover', 'troubleshoot', 'sense', 'adjust', 'operate', 'Checked', 'Feed', 'work', 'Replaced', 'lubricate', 'exhaust', 'be', 'need', 'cleane', 'inspecfted', 'monitor', 'perform', 'walk', 'check', 'brush', 'replace', 'clean', 'report', 'resolve', 'test', 'expect', 'inspect', 'find']
Verified action verbs:
['arrive', 'discover', 'troubleshoot', 'sense', 'adjust', 'operate', 'Checked', 'Feed', 'work', 'Replaced', 'lubricate', 'exhaust', 'be', 'need', 'monitor', 'perform', 'walk', 'check', 'brush', 'replace', 'clean', 'report', 'resolve', 'test', 'expect', 'inspect', 'find']
Verified noun-verb pairs:
[('work', 'be')]


In [24]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Define the text to analyze
text = """
performed pm per scope hvac tasks replaced air filters visually inspected for refrigerant leaks lubricated all bearings as needed shut off water supply  
fall visit as needed  inspected belts and adjusted as needed cleaned condensate lines checked contactors visually inspected evaporator and condenser coils 
inspected and tested heating circuit for operation inspected and tested each condensate pump if applicable refrigerations tasks inspected all hinges and 
gaskets checked electrical contacts  controls and components inspected relays and contactors visually inspected for leaks cleaned all pre filter media and 
replaced if applicable brushed coil fins as needed inspected blower wheels and fans lubricated motors and bearings as needed checked controls calibration and 
operation walk in tasks inspecfted all hinges and gaskets checked electrical contacts controls and components inspected relays and contactors visually inspected 
for leaks cleane dall pre filter media and replaced if applicable brushed coil fins as needed inspected blower wheels and fans lubricated motors and bearings 
inspected sight glass checked control calibration and operation ice machine tasks checked ice production thickness and sensing probes inspected door and gasket 
visually inspected bin and bin tstat inspected water pumps and distribution tubes visually inspected water filters inspected for any leaks cleaned condenser coils 
inspected all electrical components inspected fan motor  blades   bearings   lubricated as needed descaled ice machine exhaust fan tasks inspected roof fan belts and 
sheaves adjusted as needed replaced belts as needed  20x25x2 pleated filter high efficiency                                                                                                                                                       2023 Today  I arrived on site at unit IM01A and found that it was not operating  After troubleshooting  I discovered the following issues     Unit was off with a low water alarm    Water fill valve needs to be replaced   To resolve these issues  the following work was performed     Replaced the water fill valve    Replaced the water regulator    Checked over all operations and monitored the system   I am happy to report that the unit is now working normally  Upon departure  the unit was operating as expected  Solenoid  Water Feed 115V
"""

# Tokenize the text
tokens = word_tokenize(text)

# Perform POS tagging
tagged_tokens = pos_tag(tokens)

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to convert POS tag to WordNet POS tag
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# Extract verbs and their corresponding nouns from the POS tagged tokens
action_verbs = set()
verb_noun_pairs = []
for i, (word, pos) in enumerate(tagged_tokens):
    wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
    lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
    if wordnet_pos == wordnet.VERB:
        action_verbs.add(lemma)
        # Look for the corresponding noun following the verb
        if i < len(tagged_tokens) - 1 and tagged_tokens[i+1][1].startswith('NN'):
            verb_noun_pairs.append((lemma, tagged_tokens[i+1][0]))

# Verify that all values in action_verbs are actually verbs using nltk
def is_verb(word):
    synsets = wordnet.synsets(word, pos=wordnet.VERB)
    return bool(synsets)

# Filter out non-verbs
verified_action_verbs = [verb for verb in action_verbs if is_verb(verb)]

# Combine verified action verbs with corresponding nouns
verified_verb_noun_pairs = [(verb, noun) for verb, noun in verb_noun_pairs if verb in verified_action_verbs]

# Print the identified action verbs
print("Identified action verbs:")
print(list(action_verbs))

# Print the verified action verbs
print("Verified action verbs:")
print(verified_action_verbs)

# Print the verified verb-noun pairs
print("Verified verb-noun pairs:")
print(verified_verb_noun_pairs)

Identified action verbs:
['arrive', 'discover', 'troubleshoot', 'sense', 'adjust', 'operate', 'Checked', 'Feed', 'work', 'Replaced', 'lubricate', 'exhaust', 'be', 'need', 'cleane', 'inspecfted', 'monitor', 'perform', 'walk', 'check', 'brush', 'replace', 'clean', 'report', 'resolve', 'test', 'expect', 'inspect', 'find']
Verified action verbs:
['arrive', 'discover', 'troubleshoot', 'sense', 'adjust', 'operate', 'Checked', 'Feed', 'work', 'Replaced', 'lubricate', 'exhaust', 'be', 'need', 'monitor', 'perform', 'walk', 'check', 'brush', 'replace', 'clean', 'report', 'resolve', 'test', 'expect', 'inspect', 'find']
Verified verb-noun pairs:
[('replace', 'air'), ('need', 'shut'), ('check', 'contactors'), ('inspect', 'evaporator'), ('test', 'heating'), ('inspect', 'relays'), ('brush', 'coil'), ('lubricate', 'motors'), ('need', 'checked'), ('inspect', 'relays'), ('brush', 'coil'), ('lubricate', 'motors'), ('check', 'control'), ('sense', 'probes'), ('inspect', 'door'), ('inspect', 'bin'), ('exhau